In [ ]:
!pip install scikit-image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout,BatchNormalization,Embedding,TimeDistributed
from tensorflow.keras.layers import Conv2D,MaxPooling2D,ReLU,Concatenate,concatenate
from tensorflow.keras import regularizers,optimizers,losses
from tensorflow.keras.layers import DepthwiseConv2D,Add,ReLU,GlobalAveragePooling2D,GlobalMaxPooling2D,MultiHeadAttention
from tensorflow.keras.layers import ActivityRegularization,AvgPool2D,Conv2DTranspose
from tensorflow.keras.metrics import Accuracy,Precision,Recall,AUC,TruePositives,TrueNegatives,FalseNegatives,FalsePositives,FalseNegatives,SpecificityAtSensitivity,SensitivityAtSpecificity
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import time
import os
from glob import glob
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
import skimage.io
import skimage.color
import skimage.filters
%matplotlib inline

In [ ]:
os.listdir('/kaggle/working')

In [ ]:
images=[]
y=[]
masks=[]

gr_shape=(128,128,1)
img_shape=(128,128,3)
input_dir='/kaggle/input/lidcidri/LIDC-IDRI-slices'
out_dir='/kaggle/working'

In [ ]:
for i in range(1,5):
    img=image.load_img(f'/kaggle/input/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-000'+str(i)+'/nodule-0/images/slice-1.png')
    plt.imshow(img)
    plt.show()

In [ ]:
images=[]
y=[]
mask0=[]
mask1=[]
mask2=[]
mask3=[]
base_path='/kaggle/input/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0001'

for nodule in os.listdir(base_path):
    im='images'
    msk0='mask-0'
    msk1='mask-1'
    msk2='mask-2'
    msk3='mask-3'
    arr=[im,msk0,msk1,msk2,msk3]

    for a in arr:
        pattern=os.path.join(base_path,nodule,a,'*.png')
        files=glob(pattern)
        for filename in files:
            print(filename)
            img=image.load_img(filename,target_size=img_shape)
            img=np.asarray(img)
            if(a==im):
                images.append(img)
            elif(a==msk0):
                mask0.append(img)
            elif(a==msk1):
                mask1.append(img)
            elif(a==msk2):
                mask2.append(img)
            elif(a==msk3):
                mask3.append(img)

In [ ]:
i=5
print(mask0[i].sum())
print(mask1[i].sum())
print(mask2[i].sum())
print(mask3[i].sum())

In [ ]:
plt.imshow(mask0[i])
plt.show()

plt.imshow(mask1[i])
plt.show()

plt.imshow(mask2[i])
plt.show()

plt.imshow(mask3[i])
plt.show()

In [ ]:
gr_shape=(128,128,1)
img_shape = (128,128,3)
input_dir="/kaggle/input/lidcidri/LIDC-IDRI-slices"
images=[]
masks=[]
y=[]

In [ ]:
Thres = 5000 / 255
images = []
y = []
masks = []
input_dir = '/kaggle/input/lidcidri/LIDC-IDRI-slices'

for patient in os.listdir(input_dir):
    if len(images) > 2000:
        break
    for nodule in os.listdir(os.path.join(input_dir, patient)):
        mask_paths = {
            'mask-0': [],
            'mask-1': [],
            'mask-2': [],
            'mask-3': []
        }
        # Load images
        for filename in glob(os.path.join(input_dir, patient, nodule, 'images', '*.png')):
            img = image.load_img(filename, target_size=img_shape)
            img = np.asarray(img)
            images.append(img)
        # Load masks
        for mask_type in mask_paths.keys():
            for filename in glob(os.path.join(input_dir, patient, nodule, mask_type, '*.png')):
                img = image.load_img(filename, target_size=img_shape)
                img = np.asarray(img)
                mask_paths[mask_type].append(img)
        # Process masks
        for i in range(len(mask_paths['mask-0'])):
            white_sum = np.array([mask_paths['mask-0'][i].sum(), mask_paths['mask-1'][i].sum(),
                                  mask_paths['mask-2'][i].sum(), mask_paths['mask-3'][i].sum()])
            cnt = np.sum(white_sum > Thres)
            if cnt > 2:
                y.append(1)
                max_index = max(range(4), key=white_sum.__getitem__)
                masks.append(mask_paths[f'mask-{max_index}'][i])
            else:
                y.append(0)
                min_index = min(range(4), key=white_sum.__getitem__)
                masks.append(mask_paths[f'mask-{min_index}'][i])

In [ ]:
images=np.array(images)/255
masks=np.array(masks)/255
label=np.array(y)
print(images.shape)
print(masks.shape)
print(label.shape)
images1=images[:2000]
masks1=masks[:2000]
label1=label[:2000]
print(images1.shape)
print(masks1.shape)
print(label1.shape)
x_train,x_test,y_train,y_test,mask_train,mask_test = train_test_split(images1,label1,masks1,train_size=0.8, random_state=7)
print(x_train.shape)
print(mask_train.shape)

In [ ]:
# from tensorflow.keras.applications.resnet_v2 import ResNet152V2
# from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
# from tensorflow.keras.losses import BinaryCrossentropy

# classifier = ResNet152V2(
#             include_top = False,input_shape=img_shape,
#              weights='imagenet'
#            )
# fine_tune_at = 100
# for layer in classifier.layers[:fine_tune_at]:
#     layer.trainable = False


# Name='ResNet152V2'
# model2 = Sequential(name=Name)
# model2.add(classifier)
# model2.add(Flatten())
# model2.add(BatchNormalization())
# model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.25))
# model2.add(Dense(32, activation='sigmoid'))
# model2.add(Dropout(0.25))
# model2.add(Dense(1, activation='sigmoid'))

# print(model2.summary())
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy

# Replace these with your actual image dimensions:
# img_height, img_width, img_channels = 224, 224, 3
img_shape = x_train.shape[1:]  

# 1) Load ResNet152V2 backbone WITHOUT pre-trained weights
backbone = ResNet152V2(
    include_top=False,
    input_shape=img_shape,
    weights=None    # ← don’t attempt to download imagenet weights
)

# 2) Freeze the first `fine_tune_at` layers
fine_tune_at = 100
for layer in backbone.layers[:fine_tune_at]:
    layer.trainable = False

# 3) Build your classifier on top
Name='ResNet152V2'
model2 = Sequential(name=Name)
model2.add(backbone)
model2.add(Flatten())
model2.add(BatchNormalization())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(32, activation='sigmoid'))
model2.add(Dropout(0.25))
model2.add(Dense(1, activation='sigmoid'))

# 4) Compile (example)
model2.compile(
    optimizer='adam',
    loss=BinaryCrossentropy(),
    metrics=['accuracy']
)

# 5) Print summary
model2.summary()

In [ ]:
# Compile the model
model2.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=['acc', Recall(), Precision(), AUC(), TruePositives(), TrueNegatives(), FalseNegatives(), FalsePositives()]
)

# Plot the model architecture
plot_model(model2, to_file=Name+'.png', show_shapes=True, show_layer_names=True)

# Configure callbacks
checkpoint_filepath = 'checkpoint.weights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True,
    save_freq=5000
)

early_stopping = EarlyStopping(
    monitor='val_acc',
    patience=15,
    min_delta=0.001,
    mode='max',
    restore_best_weights=True
)

# Train the model
history = model2.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    batch_size=64,
    epochs=30,
    callbacks=[model_checkpoint_callback, early_stopping]
)

# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower left')
plt.savefig(Name+'acc.png')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(Name+'loss.png')
plt.show()

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# ------------------------------------------------------------
# 1) Predict on the test-set
# ------------------------------------------------------------
y_prob = model2.predict(x_test, batch_size=64)          # shape (N, 1)
y_pred = (y_prob > 0.5).astype(int).ravel()             # → 0 / 1

# ------------------------------------------------------------
# 2) Confusion-matrix & report
# ------------------------------------------------------------
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['Negative', 'Positive'])
fig, ax = plt.subplots(figsize=(4, 4))
disp.plot(cmap='Blues', ax=ax, colorbar=False, values_format='d')
plt.title('ResNet152V2 – Confusion Matrix')
plt.tight_layout()
plt.show()

# Save the model
model2.save(Name+'.h5')

# Save history to CSV
pd.DataFrame.from_dict(history.history).to_csv(Name+'.csv', index=False)

In [ ]:
# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower left')
plt.savefig(Name+'acc.png')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(Name+'loss.png')
plt.show()

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# ------------------------------------------------------------
# 1) Predict on the test-set
# ------------------------------------------------------------
y_prob = model2.predict(x_test, batch_size=64)          # shape (N, 1)
y_pred = (y_prob > 0.5).astype(int).ravel()             # → 0 / 1

# ------------------------------------------------------------
# 2) Confusion-matrix & report
# ------------------------------------------------------------
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['Negative', 'Positive'])
fig, ax = plt.subplots(figsize=(4, 4))
disp.plot(cmap='Blues', ax=ax, colorbar=False, values_format='d')
plt.title('ResNet152V2 – Confusion Matrix')
plt.tight_layout()
plt.show()

# Save the model
model2.save(Name+'.h5')

# Save history to CSV
pd.DataFrame.from_dict(history.history).to_csv(Name+'.csv', index=False)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers


def block(inp, filters):
    L1 = tf.keras.layers.Conv2D(filters, 1, activation='relu', padding='same', kernel_initializer='he_normal')(inp)
    L2 = tf.keras.layers.BatchNormalization()(L1)
    L3 = tf.keras.layers.Dropout(0.25)(L2)

    M1 = tf.keras.layers.Conv2D(filters, 1, activation='relu', padding='same', kernel_initializer='he_normal')(inp)
    M2 = tf.keras.layers.Conv2D(filters * 2, 3, activation='relu', padding="same")(M1)
    M3 = tf.keras.layers.Conv2D(filters, 3, activation='relu', padding='same')(M2)
    M4 = tf.keras.layers.BatchNormalization(axis=1)(M3)
    M5 = tf.keras.layers.Dropout(0.25)(M4)

    N1 = tf.keras.layers.Conv2D(filters, 1, activation='relu', padding="same", kernel_initializer='he_normal')(inp)
    N2 = tf.keras.layers.Conv2D(filters, 3, activation='relu', padding="same")(inp)
    N3 = tf.keras.layers.BatchNormalization(axis=1)(N2)

    concat = tf.keras.layers.Concatenate()([L3, M5, N3, inp])
    O1 = tf.keras.layers.Conv2D(filters * 2, 1, activation='relu', padding='same', kernel_initializer='he_normal')(concat)
    output = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(O1)
    return output

In [ ]:
Name = "CNNRegulized"
inputs = tf.keras.Input(shape=img_shape, name=Name)
A1 = layers.Conv2D(32, 3, activation="relu")(inputs)
A2 = layers.MaxPooling2D(pool_size=(2, 2))(A1)
A3 = layers.BatchNormalization()(A2)
A4 = layers.Dropout(0.25)(A3)

B1 = block(A4, 32)
B2 = block(B1, 64)
B3 = block(B2, 128)

C1 = layers.Flatten()(B3)
C2 = layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.02))(C1)
C3 = layers.Dropout(0.25)(C2)
C4 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.02))(C3)
C5 = layers.Dropout(0.25)(C4)
outputs = layers.Dense(1, activation='sigmoid')(C5)

model = tf.keras.Model(inputs, outputs, name=Name)
print(model.summary())

In [ ]:
from tensorflow.keras import losses
from tensorflow.keras.metrics import Recall, Precision, AUC, TruePositives, TrueNegatives, FalseNegatives, FalsePositives

model.compile(
    optimizer='adam',
    loss=losses.BinaryCrossentropy(),
    metrics=['acc', Recall(), Precision(), AUC(), TruePositives(), TrueNegatives(), FalseNegatives(), FalsePositives()]
)

plot_model(
    model,
    to_file=Name+'.png',
    show_shapes=True,
    show_layer_names=True
)

history = model.fit(
    x_train,
    y_train,
    epochs=30,
    validation_data=(x_test, y_test),
    batch_size=64,
)

# Plot Accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower left')
plt.savefig(Name + '_acc.png')
plt.show()

# Plot Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(Name + '_loss.png')
plt.show()

# Save Model
model.save(Name + '.h5')

# Save History
pd.DataFrame.from_dict(history.history).to_csv(Name + '.csv', index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# ------------------------------------------------------------
# 1) Predict on the test-set
# ------------------------------------------------------------
y_prob = model.predict(x_test, batch_size=64)          # shape (N, 1)
y_pred = (y_prob > 0.5).astype(int).ravel()             # → 0 / 1

# ------------------------------------------------------------
# 2) Confusion-matrix & report
# ------------------------------------------------------------
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['Negative', 'Positive'])
fig, ax = plt.subplots(figsize=(4, 4))
disp.plot(cmap='Blues', ax=ax, colorbar=False, values_format='d')
plt.title('ResNet152V2 – Confusion Matrix')
plt.tight_layout()
plt.show()

# Save the model
model2.save(Name+'.h5')

# Save history to CSV
pd.DataFrame.from_dict(history.history).to_csv(Name+'.csv', index=False)

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
img_shape = x_train.shape[1:]  
classifier = VGG19(include_top=False, input_shape=img_shape, weights='imagenet')

# Freeze layers up to a certain point
fine_tune_at = 100
for layer in classifier.layers[:fine_tune_at]:
    layer.trainable = False

Name = 'VGG19'
model3 = Sequential()
model3.add(classifier)
model3.add(Flatten())
model3.add(BatchNormalization())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(32, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(1, activation='sigmoid'))

print(model3.summary())


In [ ]:
# Compile the model
model3.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=['acc', Recall(), Precision(), AUC(), TruePositives(), TrueNegatives(), FalseNegatives(), FalsePositives()]
)

# Plot the model architecture
plot_model(model3, to_file=Name+'.png', show_shapes=True, show_layer_names=True)

# Configure callbacks
checkpoint_filepath = 'checkpoint.hdf5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True,
    save_freq=5000
)

early_stopping = EarlyStopping(
    monitor='val_acc',
    patience=15,
    min_delta=0.001,
    mode='max',
    restore_best_weights=True
)

# Train the model
history = model3.fit(
    x_train,
    y_train,
    epochs=1,
    validation_data=(x_test, y_test),
    batch_size=64,
    callbacks=[model_checkpoint_callback, early_stopping]
)

# Compile the model
model3.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=['acc', Recall(), Precision(), AUC(), TruePositives(), TrueNegatives(), FalseNegatives(), FalsePositives()]
)

# Plot the model architecture
plot_model(model3, to_file=Name+'.png', show_shapes=True, show_layer_names=True)

# Configure callbacks
checkpoint_filepath = 'checkpoint.hdf5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True,
    save_freq=5000
)

early_stopping = EarlyStopping(
    monitor='val_acc',
    patience=15,
    min_delta=0.001,
    mode='max',
    restore_best_weights=True
)

# Train the model
history = model3.fit(
    x_train,
    y_train,
    epochs=1,
    validation_data=(x_test, y_test),
    batch_size=64,
    callbacks=[model_checkpoint_callback, early_stopping]
)

# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower left')
plt.savefig(Name+'acc.png')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(Name+'loss.png')
plt.show()

# Save the model
model3.save(Name+'.h5')

#Save history to CSV
#p# # Compile the model
model3.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=['acc', Recall(), Precision(), AUC(), TruePositives(), TrueNegatives(), FalseNegatives(), FalsePositives()]
)

# Plot the model architecture
plot_model(model3, to_file=Name+'.png', show_shapes=True, show_layer_names=True)

# Configure callbacks
checkpoint_filepath = 'checkpoint.hdf5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True,
    save_freq=5000
)

early_stopping = EarlyStopping(
    monitor='val_acc',
    patience=15,
    min_delta=0.001,
    mode='max',
    restore_best_weights=True
)

# Train the model
history = model3.fit(
    x_train,
    y_train,
    epochs=1,
    validation_data=(x_test, y_test),
    batch_size=64,
    callbacks=[model_checkpoint_callback, early_stopping]
)

# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower left')
plt.savefig(Name+'acc.png')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(Name+'loss.png')
plt.show()

# Save the model
model3.save(Name+'.h5')

# Save history to CSV
pd.DataFrame.from_dict(history.history).to_csv(Name+'.csv', index=False).DataFrame.from_dict(history.history).to_csv(Name+'.csv', index=False)

In [ ]:
!pip install vit_keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from keras_hub import vit
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd

# Assuming img_shape is defined somewhere in your code
# img_shape = (height, width, channels)

# Define the ViT model
def build_vit(img_shape):
    vit_model = vit.vit_b16(

        image_size=img_shape[:2],
        activation='sigmoid',
        pretrained=True
    )

    # Build the model
    model = Sequential(name='Vision_Transformer')
    model.add(vit_model)
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(32, activation='sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
        optimizer=Adam(),
        loss=BinaryCrossentropy(),
        metrics=['acc', Precision(), Recall(), AUC(), TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()]
    )

    return model

# Build the ViT model
model2 = build_vit(img_shape)

# Print the model summary
print(model2.summary())

In [ ]:
# Configure callbacks
checkpoint_filepath = 'checkpoint.hdf5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True,
    save_freq=5000
)

early_stopping = EarlyStopping(
    monitor='val_acc',
    patience=15,
    min_delta=0.001,
    mode='max',
    restore_best_weights=True
)

# Train the model
history = model2.fit(
    x_train,
    y_train,
    epochs=30,
    validation_data=(x_test, y_test),
    batch_size=64,
    callbacks=[model_checkpoint_callback, early_stopping]
)

# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower left')
plt.savefig('accuracy.png')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('loss.png')
plt.show()

# Save the model
model2.save('model.h5')

# Save history to CSV
pd.DataFrame.from_dict(history.history).to_csv('history.csv', index=False)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower left')
plt.savefig('accuracy.png')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('loss.png')
plt.show()

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# ------------------------------------------------------------
# 1) Predict on the test-set
# ------------------------------------------------------------
y_prob = model2.predict(x_test, batch_size=64)          # shape (N, 1)
y_pred = (y_prob > 0.5).astype(int).ravel()             # → 0 / 1

# ------------------------------------------------------------
# 2) Confusion-matrix & report
# ------------------------------------------------------------
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['Negative', 'Positive'])
fig, ax = plt.subplots(figsize=(4, 4))
disp.plot(cmap='Blues', ax=ax, colorbar=False, values_format='d')
plt.title('ResNet152V2 – Confusion Matrix')
plt.tight_layout()
plt.show()

# Save the model
model2.save(Name+'.h5')

# Save history to CSV
pd.DataFrame.from_dict(history.history).to_csv(Name+'.csv', index=False)
# Save the model
model2.save('model.h5')

# Save history to CSV
pd.DataFrame.from_dict(history.history).to_csv('history.csv', index=False)